In [1]:
import os
import shutil
import logging
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import hydra
import numpy as np
import openpack_toolkit as optk
import openpack_torch as optorch
import pandas as pd
from omegaconf import DictConfig, OmegaConf

from openpack_toolkit import OPENPACK_OPERATIONS

C:\Users\Ego\anaconda3\envs\pytorchenv3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
optorch.configs.register_configs()

In [3]:
with hydra.initialize_config_dir(version_base=None, config_dir="C:\\Users\\Ego\\Documents\\TFM\\repo\\tfm1\\config"):
    cfg = hydra.compose(
        # config_name="unet.yaml",
        config_name="config.yaml",
    )    
cfg.dataset.annotation.activity_sets = dict() # Remove this attribute just for the simpler visualization.
cfg.dataset.split = optk.configs.datasets.splits.DEBUG_SPLIT

In [4]:
print(OmegaConf.to_yaml(cfg.dataset.stream))

name: all-devices
description: All devices stream.
super_stream: null
path_imu:
  dir: ${path.openpack.rootdir}/${user.name}/atr/${device}
  fname: ${session}.csv
frame_rate_imu: 30
devices:
- atr01
- atr02
- atr03
- atr04
acc: true
gyro: false
quat: false
imu_dim: 12
min_value_imu_acc: -4
max_value_imu_acc: 4
min_value_imu_gyro: -2000
max_value_imu_gyro: 2000
path_keypoint:
  dir: ${path.openpack.rootdir}/${user.name}/kinect/${..category}/${..model}/single
  fname: ${session}.json
frame_rate_keypoint: 15
category: 2d-kpt
model: mmpose-hrnet-w48-posetrack18-384x288-posewarper-stage2
nodes:
  0: nose
  1: left_eye
  2: right_eye
  3: left_ear
  4: right_ear
  5: left_shoulder
  6: right_shoulder
  7: left_elbow
  8: right_elbow
  9: left_wrist
  10: right_wrist
  11: left_hip
  12: right_hip
  13: left_knee
  14: right_knee
  15: left_ankle
  16: right_ankle
keypoint_dim: 34
min_value_keypoint: 0
max_value_keypoint: 1280



In [5]:
import sys
sys.path.append('../')
from utils.datasets import OpenPackAll
class OpenPackAllDataModule(optorch.data.OpenPackBaseDataModule):
    dataset_class = OpenPackAll

    def get_kwargs_for_datasets(self, stage: Optional[str] = None) -> Dict:
        kwargs = {
            "window": self.cfg.train.window,
            "debug": self.cfg.debug,
        }
        return kwargs


In [6]:
datamodule = OpenPackAllDataModule(cfg)
datamodule.setup("test")
dataloaders = datamodule.test_dataloader()

batch = dataloaders[0].dataset.__getitem__(0)
print(batch.keys())

Shape of x keypoints: (2, 27377, 17)
Shape of t keypoints: (27377,)
Shape of x keypoints: (27377, 34)
Shape of x imu: (55272, 12)
Shape of t imu: (55272,)
First t imu entries [1634885785026 1634885785059 1634885785092 ... 1634887608903 1634887608936
 1634887608969]
First t keypoints entries [1634885784059 1634885784126 1634885784192 ... 1634887609837 1634887609903
 1634887609970]
(55272, 47)
      0_x     1_x     2_x     3_x     4_x     5_x     6_x     7_x     8_x  \
0  0.5318  0.7782  0.2276  0.3159 -0.7739  0.1986 -0.1385  1.0100 -0.0305   
1  0.5589  0.7714  0.2073  0.3132 -0.8086  0.2437 -0.1353  0.9929 -0.0319   
2  0.5680  0.7787  0.1800  0.3144 -0.8562  0.2772 -0.1356  0.9812 -0.0478   
3  0.5670  0.8324  0.2005  0.2864 -0.8574  0.3009 -0.1397  0.9670 -0.0385   
4  0.5946  0.8991  0.2110  0.2654 -0.8530  0.3531 -0.1497  0.9704 -0.0307   

      9_x  ...     24     25     26     27     28     29     30     31     32  \
0  0.1779  ...  572.3  596.8  629.0  676.9  569.1  676.9  640

No preprocessing is applied.


IMU time 0        0
1        0
2        0
3        0
4        0
        ..
55267    0
55268    0
55269    0
55270    0
55271    0
Name: annot_time, Length: 55272, dtype: int64
Keypoints time 0        0
1        0
2        0
3        0
4        0
        ..
27372    0
27373    0
27374    0
27375    0
27376    0
Name: annot_time, Length: 27377, dtype: int64
Shape of the merged array: (55272, 46)
dict_keys(['x', 't', 'ts'])


SyntaxError: invalid syntax (3628055297.py, line 1)